### LLMChains & OutputParsers

Instead of just using models you can combine a model and a prompt. This can be done with the LLMChain class.
We will additional, more complex chains in this notebook

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment? Sentiment is required.
subject: What subject is the text about? Use exactly one word. Use 'None' if no subject was provided.
price: How much did the customer pay? Use 'None' if no price was provided.

Format the output as JSON with the following keys:
sentiment
subject
price

text: {input}
"""

In [3]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain
from pydantic import SecretStr

llm = ChatOpenAI( 
  model=os.getenv("MODEL", ""),
  base_url=os.getenv("API_URL", ""),
  api_key=SecretStr(os.getenv("API_KEY", "")),
  temperature=0
)

prompt_template = ChatPromptTemplate.from_template(template=TEMPLATE)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input={"input": "I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp."})

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_16588\621871601.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


{'input': 'I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.',
 'text': '{\n  "sentiment": "neutral",\n  "subject": "pizza",\n  "price": "None"\n}'}

### Response Schemas

There were two issues with the output: The output also contains text and the output is just a string, not a dictionary.

In [4]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="sentiment", description="is the text in a positive, neutral or negative sentiment? Sentiment is required."),
    ResponseSchema(name="subject", description="What subject is the text about? Use exactly one word. Use None if no price was provided."),
    ResponseSchema(name="price", description="How much did the customer pay? Use None if no price was provided.", type="float")
]
print(response_schemas)

[ResponseSchema(name='sentiment', description='is the text in a positive, neutral or negative sentiment? Sentiment is required.', type='string'), ResponseSchema(name='subject', description='What subject is the text about? Use exactly one word. Use None if no price was provided.', type='string'), ResponseSchema(name='price', description='How much did the customer pay? Use None if no price was provided.', type='float')]


In [5]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentiment": string  // is the text in a positive, neutral or negative sentiment? Sentiment is required.
	"subject": string  // What subject is the text about? Use exactly one word. Use None if no price was provided.
	"price": float  // How much did the customer pay? Use None if no price was provided.
}
```


In [6]:
# Create prompt template
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Interprete the text and evaluate the text. "
            "sentiment: is the text in a positive, neutral or negative sentiment? "
            "subject: What subject is the text about? Use exactly one word. "
            "Just return the JSON, do not add ANYTHING, NO INTERPRETATION! "
            "text: {input}\n"
            "{format_instructions}\n"
        )
    ],
    input_variables=["input"],
    partial_variables={"format_instructions": format_instructions}
)

In [7]:
_input = prompt.format_prompt(input="I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.")
output = llm.invoke(_input.to_messages())
print(output.content)

```json
{
	"sentiment": "neutral",
	"subject": "pizza",
	"price": null
}
```



In [8]:
json_output = parser.parse(str(output.content))
print(json_output)

{'sentiment': 'neutral', 'subject': 'pizza', 'price': None}


In [9]:
json_output.get("sentiment")

'neutral'